1. Import necessary libraries and modules


In [ ]:
import pandas as pd
import numpy as np
import urllib.request
import zipfile
import os
import glob
from datetime import datetime
import time

Decorator

In [30]:
def time_se(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"Function '{func.__name__}': {end - start:.4f} sec")
        return result
    return wrapper


2. Download data, syys




In [14]:
check_files = glob.glob("data/household_power_consumption.txt")
if check_files:
    print("You already have data")
else:
    print("Wait, data downloading...")
    dir_path = "./data"
    data_filename = "data/dataset_ez.zip"
    data_url = "https://archive.ics.uci.edu/static/public/235/individual+household+electric+power+consumption.zip" 

    os.makedirs(dir_path, exist_ok=True)
    urllib.request.urlretrieve(data_url, data_filename)

    with zipfile.ZipFile(data_filename, 'r') as zip:
        zip.extractall(dir_path)
    print("DONE!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")


You already have data


3. Creating pandas dataframe and numpy array )

In [15]:
file_path = "data/household_power_consumption.txt"
dtype_df = {
    'Global_active_power': 'float',
    'Global_reactive_power': 'float',
    'Voltage': 'float',
    'Global_intensity': 'float',
    'Sub_metering_1': 'float',
    'Sub_metering_2': 'float',
    'Sub_metering_3': 'float',
}

#DF---------------------------------------------------------------------------------------------------
@time_se
def DataFrame():
    df = pd.read_csv(file_path, sep=";", dtype=dtype_df, na_values='?')
    df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
    df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S')
    df = df.dropna()
    return df

#ARR--------------------------------------------------------------------------------------------------
@time_se
def NpArray():
    arr = np.genfromtxt(file_path, delimiter=';', names=True, dtype = None, missing_values ="?" )
    numeric_coll = [name for name in arr.dtype.names if np.issubdtype(arr.dtype[name], np.number)]
    mask = ~np.any([np.isnan(arr[field]) for field in numeric_coll], axis=0)  
    arr = arr[mask]
    

    return arr







<span style="font-size: 24px;"><b>TASKS</b></span>


1. Обрати всі записи, у яких загальна активна споживана потужність
перевищує 5 кВт.


In [16]:
#DF--------------------------------------------------------------------------------
@time_se
def task1_df(df):
    taska1_df = df[df['Global_active_power'] > 5]
    return taska1_df

#ARR-------------------------------------------------------------------------------
@time_se
def task1_arr(arr):
    taska1_arr = arr[arr['Global_active_power'] > 5]
    return taska1_arr

2. Обрати всі записи, у яких вольтаж перевищує 235 В.

In [17]:
#DF--------------------------------------------------------------------------------
@time_se
def task2_df(df):
    taska2_df = df[df['Voltage'] > 235]
    return taska2_df

#ARR-------------------------------------------------------------------------------
@time_se
def task2_arr(arr):
    taska2_arr = arr[arr['Voltage'] > 235]
    return taska2_arr


3. Обрати всі записи, у яких сила струму лежить в межах 19-20 А, для них виявити ті, у яких пральна машина та холодильник споживають більше, ніж бойлер та кондиціонер.

In [18]:
#DF--------------------------------------------------------------------------------
@time_se
def task3_df(df):
    taska3_df = df[(df['Global_intensity'] >= 19) & (df['Global_intensity'] <= 20) & (df['Sub_metering_2'] > df['Sub_metering_3'])]
    return taska3_df

#ARR-------------------------------------------------------------------------------
@time_se
def task3_arr(arr):
    taska3_arr = arr[(arr['Global_intensity'] >= 19) & (arr['Global_intensity'] <= 20) & (arr['Sub_metering_2'] > arr['Sub_metering_3'])]
    return taska3_arr


4. Обрати випадковим чином 500000 записів (без повторів елементів вибірки), для них обчислити середні величини усіх 3-х груп споживання електричної енергії

In [ ]:
#DF--------------------------------------------------------------------------------
@time_se
def task4_df(df, x):
    random_df = df.sample(n=500000, replace=False, random_state=x)
    avg_df = random_df[['Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']].mean()
    return random_df, avg_df  

#ARR-------------------------------------------------------------------------------
@time_se
def task4_arr(arr):
    random_rows = np.random.choice(len(arr), size=500000, replace=False)
    random_arr = arr[random_rows]
    avg_arr = []
    for name in ['Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']:
        avg_arr.append(np.mean(random_arr[name]))
    return random_arr, avg_arr



Function 'DataFrame':  3.9746 sec
Function 'task4_df': 0.1031 sec


(              Date                Time  Global_active_power  \
 1030580 2008-12-01 1900-01-01 09:44:00                1.502   
 1815    2006-12-17 1900-01-01 23:39:00                0.374   
 1295977 2009-06-03 1900-01-01 17:01:00                0.620   
 206669  2007-05-09 1900-01-01 05:53:00                0.280   
 1048893 2008-12-14 1900-01-01 02:57:00                1.372   
 ...            ...                 ...                  ...   
 229152  2007-05-24 1900-01-01 20:36:00                0.302   
 1328200 2009-06-26 1900-01-01 02:04:00                0.212   
 592479  2008-02-01 1900-01-01 04:03:00                0.414   
 1492572 2009-10-18 1900-01-01 05:36:00                0.402   
 430715  2007-10-11 1900-01-01 19:59:00                3.866   
 
          Global_reactive_power  Voltage  Global_intensity  Sub_metering_1  \
 1030580                  0.074   240.17               6.4             0.0   
 1815                     0.264   245.50               1.8             0.0

5. Обрати ті записи, які після 18-00 споживають понад 6 кВт за хвилину в середньому, серед відібраних визначити ті, у яких основне споживання електроенергії у вказаний проміжок часу припадає на пральну машину, сушарку, холодильник та освітлення (група 2 є найбільшою), а потім обрати кожен третій результат із першої половини та кожен четвертий результат із другої половини.

In [20]:
#DF--------------------------------------------------------------------------------
@time_se
def task5_df(df):
    task_time = pd.to_datetime('18:00:00', format='%H:%M:%S').time()
    after_6 = df[(df['Time'].dt.time >= task_time) & (df['Global_active_power'] > 6)]

    domination_gr2 = after_6.query('Sub_metering_2 > Sub_metering_1 and Sub_metering_2 > Sub_metering_3')

    polovina = len(domination_gr2) // 2
    polovina_1 =  domination_gr2.iloc[:polovina]
    polovina_2 =  domination_gr2.iloc[polovina:]
    polovina_1_3 = polovina_1.iloc[::3]
    polovina_2_4 = polovina_2.iloc[::4]
    taska_5 = pd.concat([polovina_1_3, polovina_2_4])
    return taska_5
    
#ARR-------------------------------------------------------------------------------
@time_se
def task5_arr(arr):
    task_time = datetime.strptime("18:00:00", "%H:%M:%S").time()
    Date_Time = np.char.add(np.char.add(arr['Date'], ' '), arr['Time'])
    arr_Date_Time = [datetime.strptime(dt, "%d/%m/%Y %H:%M:%S") for dt in Date_Time]
    arr_Time = np.array([dt.time() for dt in arr_Date_Time])
    after_6 = arr[arr_Time > task_time]

    domination_gr2 = after_6[
        (after_6['Sub_metering_2'] > after_6['Sub_metering_3']) & 
        (after_6['Sub_metering_2'] > after_6['Sub_metering_1'])
        ]

    polovina = len(domination_gr2) // 2
    polovina_1 = domination_gr2[:polovina]
    polovina_2 = domination_gr2[polovina:]
    polovina_1_3 = polovina_1[::3]
    polovina_2_4 = polovina_2[::4]  
    taska_5 = np.concatenate((polovina_1_3, polovina_2_4))
    return taska_5



<span style = 'font-size: 24px;'><b>Testing<b></span>

In [26]:
df = DataFrame()
arr = NpArray()

t5arr = task5_arr(arr)
t5df = task5_df(df)


Function 'DataFrame':  4.2687 sec
Function 'NpArray':  9.9752 sec
Function 'task5_arr':  10.3673 sec
Function 'task5_df':  0.4353 sec
